In [1]:
import os
import errno
import numpy as np 
import deepcell
from deepcell_toolbox.multiplex_utils import multiplex_preprocess

In [2]:
# create folder for this set of experiments
experiment_folder = "size_benchmarking"
MODEL_DIR = os.path.join("/data/analyses", experiment_folder)
NPZ_DIR = "/data/npz_data/20201018_freeze/"
LOG_DIR = '/data/logs'

if not os.path.isdir(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [10]:
from deepcell.utils.data_utils import get_data
from skimage.segmentation import relabel_sequential

npz_name = "20201018_multiplex_seed_2_"

train_dict = np.load(NPZ_DIR + npz_name + "train_512x512_split.npz", allow_pickle=True)
val_dict = np.load(NPZ_DIR + npz_name + "val_256x256_split.npz", allow_pickle=True)

In [11]:
train_keys, val_keys = list(train_dict.keys()), list(val_dict.keys())
print(train_keys)

['1', '3', '10', '33', '100', '333', '1000', '2665']


In [12]:
from deepcell.model_zoo.panopticnet import PanopticNet
from tensorflow.keras.optimizers import SGD, Adam
from deepcell.utils.train_utils import rate_scheduler
from deepcell import image_generators
from deepcell.utils import train_utils
from tensorflow.python.keras.losses import MSE
from deepcell import losses
from deepcell.utils.train_utils import get_callbacks
from deepcell.utils.train_utils import count_gpus

for idx in range(6, len(train_keys)):
    print("Training model for {}".format(train_keys[idx]))
    train_key, val_key = train_keys[idx], val_keys[idx]
    
    # initialize new model
    new_model = PanopticNet(
        backbone='resnet50',
        input_shape=(256, 256, 2),
        norm_method=None,
        num_semantic_heads=2,
        num_semantic_classes=[1, 3], # inner distance, pixelwise
        location=True,  # should always be true
        include_top=True)
    
    
    X_train = train_dict[train_key].item()['X']
    X_train = multiplex_preprocess(X_train)
    y_train = train_dict[train_key].item()['y']
    print("X_train shape is {}, y_train shape is {}".format(X_train.shape, y_train.shape))
    
    
    X_val = val_dict[val_key].item()['X']
    X_val = multiplex_preprocess(X_val)
    y_val = val_dict[val_key].item()['y']
    print("X_val shape is {}, y_val shape is {}".format(X_val.shape, y_val.shape))
        
    # set up training parameters
    model_name = npz_name + '_subset_' + train_key
    n_epoch = 100  # Number of training epochs

    optimizer = Adam(lr=1e-4, clipnorm=0.001)
    lr_sched = rate_scheduler(lr=1e-4, decay=0.99)

    batch_size = 8

    min_objects = 0  # throw out images with fewer than this many objects
    seed=0
    print("Model name is {}".format(model_name))
    
    # create augmented dataset
    datagen = image_generators.CroppingDataGenerator(
        rotation_range=180,
        shear_range=0,
        zoom_range=(0.7, 1/0.7),
        horizontal_flip=True,
        vertical_flip=True,
        crop_size=(256, 256),
        float_dtype='float16',
        int_dtype='int16')

    datagen_val = image_generators.SemanticDataGenerator(
        rotation_range=0,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=0,
        vertical_flip=0,
        float_dtype='float16',
        int_dtype='int16')

    train_data = datagen.flow(
        {'X': X_train, 'y': y_train},
        seed=seed,
        transforms=['inner-distance', 'pixelwise'],
        transforms_kwargs={'pixelwise':{'dilation_radius': 1}, 
                          'inner-distance': {'erosion_width': 1, 'alpha': 'auto'}},
        min_objects=min_objects,
        batch_size=batch_size)

    val_data = datagen_val.flow(
        {'X': X_val, 'y': y_val},
        seed=seed,
        transforms=['inner-distance', 'pixelwise'],
        transforms_kwargs={'pixelwise':{'dilation_radius': 1},
                          'inner-distance': {'erosion_width': 1, 'alpha': 'auto'}},
        min_objects=min_objects,
        batch_size=batch_size)
    
    print('generators created')
    
    # set up losses
    def semantic_loss(n_classes):
        def _semantic_loss(y_pred, y_true):
            if n_classes > 1:
                return 0.01 * losses.weighted_categorical_crossentropy(
                    y_pred, y_true, n_classes=n_classes)
            return MSE(y_pred, y_true)
        return _semantic_loss


    loss = {}

    # Give losses for all of the semantic heads
    for layer in new_model.layers:
        if layer.name.startswith('semantic_'):
            n_classes = layer.output_shape[-1]
            loss[layer.name] = semantic_loss(n_classes)
            
    # compile model
    new_model.compile(loss=loss, optimizer=optimizer)
    
    # train model
    model_path = os.path.join(MODEL_DIR, '{}.h5'.format(model_name))
    loss_path = os.path.join(MODEL_DIR, '{}.npz'.format(model_name))

    num_gpus = count_gpus()

    print('Training on', num_gpus, 'GPUs.')

    train_callbacks = get_callbacks(
        model_path,
        lr_sched=lr_sched,
        #tensorboard_log_dir=LOG_DIR,
        save_weights_only=num_gpus >= 2,
        monitor='val_loss',
        verbose=1)

    loss_history = new_model.fit_generator(
        train_data,
        steps_per_epoch=333,
        epochs=n_epoch,
        validation_data=val_data,
        validation_steps=val_data.y.shape[0] // batch_size,
        callbacks=train_callbacks)


Training model for 1000
X_train shape is (1000, 512, 512, 2), y_train shape is (1000, 512, 512, 1)


W1027 04:30:31.265104 139858664515392 semantic.py:111] X data dtype is float32: this will increase memory use during preprocessing. Consider using a smaller dtype


X_val shape is (1230, 256, 256, 2), y_val shape is (1230, 256, 256, 1)
Model name is 20201018_multiplex_seed_2__subset_1000


W1027 04:33:14.317371 139858664515392 semantic.py:111] X data dtype is float32: this will increase memory use during preprocessing. Consider using a smaller dtype


generators created
Training on 1 GPUs.
Epoch 1/100
332/333 [============================>.] - ETA: 0s - loss: 0.0810 - semantic_0_loss: 0.0735 - semantic_1_loss: 0.0075
Epoch 00001: val_loss improved from inf to 0.15149, saving model to /data/analyses/size_benchmarking/20201018_multiplex_seed_2__subset_1000.h5
333/333 [==============================] - 958s 3s/step - loss: 0.0808 - semantic_0_loss: 0.0733 - semantic_1_loss: 0.0075 - val_loss: 0.1515 - val_semantic_0_loss: 0.0513 - val_semantic_1_loss: 0.1001
Epoch 2/100
332/333 [============================>.] - ETA: 0s - loss: 0.0251 - semantic_0_loss: 0.0188 - semantic_1_loss: 0.0062
Epoch 00002: val_loss improved from 0.15149 to 0.05394, saving model to /data/analyses/size_benchmarking/20201018_multiplex_seed_2__subset_1000.h5
333/333 [==============================] - 89s 268ms/step - loss: 0.0251 - semantic_0_loss: 0.0188 - semantic_1_loss: 0.0062 - val_loss: 0.0539 - val_semantic_0_loss: 0.0366 - val_semantic_1_loss: 0.0174
Epoch

...

332/333 [============================>.] - ETA: 0s - loss: 0.0142 - semantic_0_loss: 0.0098 - semantic_1_loss: 0.0044
Epoch 00061: val_loss did not improve from 0.01247
333/333 [==============================] - 98s 293ms/step - loss: 0.0142 - semantic_0_loss: 0.0098 - semantic_1_loss: 0.0044 - val_loss: 0.1509 - val_semantic_0_loss: 0.1470 - val_semantic_1_loss: 0.0039
Epoch 62/100
332/333 [============================>.] - ETA: 0s - loss: 0.0139 - semantic_0_loss: 0.0096 - semantic_1_loss: 0.0043
Epoch 00062: val_loss improved from 0.01247 to 0.01242, saving model to /data/analyses/size_benchmarking/20201018_multiplex_seed_2__subset_2665.h5
333/333 [==============================] - 104s 313ms/step - loss: 0.0139 - semantic_0_loss: 0.0096 - semantic_1_loss: 0.0043 - val_loss: 0.0124 - val_semantic_0_loss: 0.0086 - val_semantic_1_loss: 0.0039
Epoch 63/100
332/333 [============================>.] - ETA: 0s - loss: 0.0141 - semantic_0_loss: 0.0098 - semantic_1_loss: 0.0043
Epoch 00063: 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



332/333 [============================>.] - ETA: 0s - loss: 0.0136 - semantic_0_loss: 0.0093 - semantic_1_loss: 0.0043
Epoch 00080: val_loss did not improve from 0.01221
333/333 [==============================] - 106s 318ms/step - loss: 0.0136 - semantic_0_loss: 0.0093 - semantic_1_loss: 0.0043 - val_loss: 0.0143 - val_semantic_0_loss: 0.0098 - val_semantic_1_loss: 0.0045
Epoch 81/100
332/333 [============================>.] - ETA: 0s - loss: 0.0137 - semantic_0_loss: 0.0094 - semantic_1_loss: 0.0043
Epoch 00081: val_loss did not improve from 0.01221
333/333 [==============================] - 107s 321ms/step - loss: 0.0137 - semantic_0_loss: 0.0094 - semantic_1_loss: 0.0043 - val_loss: 0.1713 - val_semantic_0_loss: 0.1670 - val_semantic_1_loss: 0.0043
Epoch 82/100
332/333 [============================>.] - ETA: 0s - loss: 0.0137 - semantic_0_loss: 0.0094 - semantic_1_loss: 0.0043
Epoch 00082: val_loss did not improve from 0.01221
333/333 [==============================] - 106s 319ms/ste

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



332/333 [============================>.] - ETA: 0s - loss: 0.0134 - semantic_0_loss: 0.0092 - semantic_1_loss: 0.0042
Epoch 00099: val_loss did not improve from 0.01210
333/333 [==============================] - 107s 322ms/step - loss: 0.0134 - semantic_0_loss: 0.0092 - semantic_1_loss: 0.0042 - val_loss: 0.0130 - val_semantic_0_loss: 0.0087 - val_semantic_1_loss: 0.0043
Epoch 100/100
332/333 [============================>.] - ETA: 0s - loss: 0.0135 - semantic_0_loss: 0.0093 - semantic_1_loss: 0.0042
Epoch 00100: val_loss did not improve from 0.01210
333/333 [==============================] - 107s 321ms/step - loss: 0.0135 - semantic_0_loss: 0.0093 - semantic_1_loss: 0.0042 - val_loss: 0.0127 - val_semantic_0_loss: 0.0085 - val_semantic_1_loss: 0.0042
